In [ ]:
import pandas as pd
import regex as re

pd.set_option('display.max_colwidth', None)
file = '/content/sample_data/who_2020.xlsx'
df = pd.read_excel(file, sheet_name='Annex 2-2', header=1)
delete = list(range(5,20))
df=df.drop(df.columns[delete], axis=1)
df.drop(index=[0,1], inplace=True)
df=df.rename(columns={'Unnamed: 0':'Country'})
df.drop(index=[2], inplace=True)
df.set_index('Country')
df=df[~df['Country'].str.contains('Region', flags=re.I, regex=True, na=False)]
df.drop(index=[197,205], inplace=True)
df.rename(columns={
    df.columns[1]: '% probability of dying bewteen 30 and 70 of illness',
    df.columns[2]: 'Suicide rate per per 100 000',
    df.columns[3]: 'Alcohol Consumption (litres)',
    df.columns[4]: 'Road traffic mortality per 100 000'}, inplace=True)
df.replace('<0.1','0.1',inplace=True)
df.replace(' ','',inplace=True)
df.replace('-','',inplace=True)
df['Alcohol Consumption (litres)']=pd.to_numeric(df['Alcohol Consumption (litres)'], errors='coerce')
df=df.sort_values('Alcohol Consumption (litres)', ascending=False)

In [ ]:
import requests
from bs4 import BeautifulSoup

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_European_Union_member_states_by_population').text
soup = BeautifulSoup(website_url,'lxml')

my_table = soup.find('table',{'class':'sortable wikitable'})
links = my_table.findAll('a')

countries = []
for link in links:
  countries.append(link.get('title'))

dfc = pd.DataFrame()
dfc['countries'] = countries
u = dfc['countries'].unique()

dfd = pd.DataFrame(u)
dfd.rename(columns={'0':'Country'}, inplace=True)
dfd.drop(index=0, inplace=True)
dfd.columns=['Country']
dfd.at[19, 'Country'] = 'Ireland'

In [ ]:
df['EU?']=df['Country'].isin(dfd['Country'])
df_eu = df[df['EU?']==True]

In [167]:
import plotly.express as px
px.bar(df_eu, x='Country', y='Alcohol Consumption (litres)', text='Alcohol Consumption (litres)').show()